In [2]:
# I decided to treat this as a classification problem by creating a new binary variable affair
# (did the woman have at least one affair?) and trying to predict the classification for each
# woman.

# Dataset

# The dataset I chose is the affairs dataset that comes with Statsmodels. It was derived
# from a survey of women in 1974 by Redbook magazine, in which married women were
# asked about their participation in extramarital affairs. More information about the study
# is available in a 1978 paper from the Journal of Political Economy.

# Description of Variables
# The dataset contains 6366 observations of 9 variables:
# rate_marriage: woman's rating of her marriage (1 = very poor, 5 = very good)
# age: woman's age
# yrs_married: number of years married
# children: number of children
# religious: woman's rating of how religious she is (1 = not religious, 4 = strongly religious)
# educ: level of education (9 = grade school, 12 = high school, 14 = some college, 16 = # college graduate, 17 = some graduate school, 20 = advanced degree)
# occupation: woman's occupation (1 = student, 2 = farming/semi-skilled/unskilled, 3 = "white collar", # 4 = teacher/nurse/writer/technician/skilled, 5 = managerial/business, 6 = professional with advanced degree)
# occupation_husb: husband's occupation (same coding as above)
# affairs: time spent in extra-marital affairs

# Code to loading data and modules
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection  import cross_val_score
dta = sm.datasets.fair.load_pandas().data

In [3]:
dta.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0.111111
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3.230769
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1.400000
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0.727273
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4.666666


In [6]:
# add "affair" column: 1 represents having affairs, 0 represents not
dta['affair'] = (dta.affairs > 0).astype(int)
y, X = dmatrices('affair ~ rate_marriage + age + yrs_married + children + \
religious + educ + C(occupation) + C(occupation_husb)',
dta, return_type="dataframe")

X = X.rename(columns = {'C(occupation)[T.2.0]':'occ_2',
'C(occupation)[T.3.0]':'occ_3',
'C(occupation)[T.4.0]':'occ_4',
'C(occupation)[T.5.0]':'occ_5',
'C(occupation)[T.6.0]':'occ_6',
'C(occupation_husb)[T.2.0]':'occ_husb_2',
'C(occupation_husb)[T.3.0]':'occ_husb_3',
'C(occupation_husb)[T.4.0]':'occ_husb_4',
'C(occupation_husb)[T.5.0]':'occ_husb_5',
'C(occupation_husb)[T.6.0]':'occ_husb_6'})
y = np.ravel(y)
y

array([1., 1., 1., ..., 0., 0., 0.])

In [7]:
dta.isnull().sum()

rate_marriage      0
age                0
yrs_married        0
children           0
religious          0
educ               0
occupation         0
occupation_husb    0
affairs            0
affair             0
dtype: int64

In [8]:
dta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6366 entries, 0 to 6365
Data columns (total 10 columns):
rate_marriage      6366 non-null float64
age                6366 non-null float64
yrs_married        6366 non-null float64
children           6366 non-null float64
religious          6366 non-null float64
educ               6366 non-null float64
occupation         6366 non-null float64
occupation_husb    6366 non-null float64
affairs            6366 non-null float64
affair             6366 non-null int32
dtypes: float64(9), int32(1)
memory usage: 472.6 KB


In [10]:
dta['affairs'] = (dta['affairs']>0).astype(int)
dta.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs,affair
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,1,1
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,1,1
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1,1
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,1,1
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,1,1


In [17]:
y, X = dmatrices('affair ~ rate_marriage + age + yrs_married + children + religious + educ + C(occupation) + C(occupation_husb)', dta, return_type="dataframe")

In [23]:
X = X.rename(columns = {'C(occupation)[T.2.0]':'occ_2',
'C(occupation)[T.3.0]':'occ_3',
'C(occupation)[T.4.0]':'occ_4',
'C(occupation)[T.5.0]':'occ_5',
'C(occupation)[T.6.0]':'occ_6',
'C(occupation_husb)[T.2.0]':'occ_husb_2',
'C(occupation_husb)[T.3.0]':'occ_husb_3',
'C(occupation_husb)[T.4.0]':'occ_husb_4',
'C(occupation_husb)[T.5.0]':'occ_husb_5',
'C(occupation_husb)[T.6.0]':'occ_husb_6'})
y = np.ravel(y)

In [25]:
y

array([1., 1., 1., ..., 0., 0., 0.])

In [27]:
# fit model
logit = sm.Logit(y, X)
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.542911
         Iterations 6


In [28]:
# model summary
result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Logit
==================================================================
Model:              Logit            Pseudo R-squared: 0.137      
Dependent Variable: y                AIC:              6946.3465  
Date:               2019-04-22 22:17 BIC:              7061.2449  
No. Observations:   6366             Log-Likelihood:   -3456.2    
Df Model:           16               LL-Null:          -4002.5    
Df Residuals:       6349             LLR p-value:      1.5339e-222
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     6.0000                                        
------------------------------------------------------------------
                   Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
------------------------------------------------------------------
Intercept          2.9708   0.5722   5.1917 0.0000  1.8492  4.0923
occ_2              0.3902   0.4476   0.8719 0.3832 -0.4869  1.2674
occ_3              0.7027   0.4415   1.5917 0.1114 -0.1626  1.5679
occ_4              0.4714   0.4425   1.0652 0.2868 -0.3959  1.3387
occ_5              1.0542   0.4466   2.3603 0.0183  0.1788  1.9296
occ_6              1.1080   0.4942   2.2420 0.0250  0.1394  2.0767
occ_husb_2         0.1704   0.1861   0.9160 0.3597 -0.1943  0.5352
occ_husb_3         0.2842   0.2022   1.4057 0.1598 -0.1121  0.6804
occ_husb_4         0.1428   0.1810   0.7892 0.4300 -0.2119  0.4976
occ_husb_5         0.1723   0.1826   0.9436 0.3454 -0.1856  0.5303
occ_husb_6         0.1828   0.2037   0.8974 0.3695 -0.2164  0.5819
rate_marriage     -0.7102   0.0315 -22.5600 0.0000 -0.7719 -0.6485
age               -0.0613   0.0103  -5.9361 0.0000 -0.0815 -0.0410
yrs_married        0.1080   0.0110   9.8364 0.0000  0.0865  0.1295
children           0.0156   0.0321   0.4881 0.6255 -0.0472  0.0785
religious         -0.3754   0.0349 -10.7657 0.0000 -0.4437 -0.3070
educ              -0.0017   0.0174  -0.0992 0.9210 -0.0358  0.0324
==================================================================

"""

In [30]:
# fit model
model = LogisticRegression()
model = model.fit(X, y)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [32]:
# Model accuracy
model.score(X, y)

0.7258875274897895

In [33]:
y.mean()

0.3224945020420987

In [35]:
print(model.coef_)
print(model.intercept_)

[[ 1.48983589  0.18806639  0.49894787  0.25066856  0.83900806  0.83390843
   0.19063594  0.29783271  0.16140885  0.18777091  0.19401637 -0.70312336
  -0.05841777  0.10567654  0.01691927 -0.37113627  0.0040165 ]]
[1.48983589]


In [37]:
# examine the coefficients
for el in zip(X.columns, np.transpose(model.coef_).tolist()):
    print(el)

('Intercept', [1.489835891324933])
('occ_2', [0.18806639024440983])
('occ_3', [0.4989478668156914])
('occ_4', [0.25066856498524825])
('occ_5', [0.8390080648117001])
('occ_6', [0.8339084337443315])
('occ_husb_2', [0.1906359445867889])
('occ_husb_3', [0.2978327129263421])
('occ_husb_4', [0.1614088540760616])
('occ_husb_5', [0.18777091388972483])
('occ_husb_6', [0.19401637225511495])
('rate_marriage', [-0.7031233597323255])
('age', [-0.05841777448168919])
('yrs_married', [0.10567653799735635])
('children', [0.016919266970905608])
('religious', [-0.3711362653137546])
('educ', [0.00401650319563816])


In [39]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [41]:
x_train.shape, x_test.shape, y_test.shape, y_test.shape

((4456, 17), (1910, 17), (1910,), (1910,))

In [51]:
y_pred = model.predict(x_test)
y_pred

array([1., 0., 0., ..., 0., 0., 0.])

In [50]:
y_prob = model.predict_proba(x_test)
y_prob

array([[0.34304301, 0.65695699],
       [0.90808777, 0.09191223],
       [0.7384556 , 0.2615444 ],
       ...,
       [0.58000259, 0.41999741],
       [0.82206596, 0.17793404],
       [0.75523253, 0.24476747]])

In [52]:
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.roc_auc_score(y_test, y_prob[:, 1]))

0.7324607329842932
0.7492935451201826


In [62]:
scores = cross_val_score(LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=5000), X, y, scoring='accuracy', cv=10)
scores, scores.mean()

(array([0.71943574, 0.70219436, 0.73981191, 0.70597484, 0.70440252,
        0.73113208, 0.73427673, 0.70440252, 0.75314465, 0.74685535]),
 0.7241630685514876)

In [55]:
model.predict_proba(np.array([[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 3, 25, 3, 1, 4, 16]]))

array([[0.77472221, 0.22527779]])